# 微分可能LUTモデルによるMNIST学習

Stochasticモデルに BatchNormalization や Binarize(backward時はHard-Tanh)を加えることで、より一般的なデータに対してLUT回路学習を行います。 

## 事前準備

In [13]:
import os
import shutil
import numpy as np
#from tqdm.notebook import tqdm
from tqdm import tqdm

import binarybrain as bb

import torch
import torchvision
import torchvision.transforms as transforms

device_id = 0
bb.set_device(device_id)
print(f"BinaryBrain : {bb.get_version_string()}")
print(f"Device : {bb.get_device_name(device_id)}")

BinaryBrain : 4.3.2
Device : NVIDIA GeForce GTX 1660 SUPER


異なる閾値で2値化した画像でフレーム数を水増ししながら学習させます。この水増しをバイナリ変調と呼んでいます。

ここではフレーム方向の水増し量を frame_modulation_size で指定しています。

In [14]:
# configuration
data_path             = './data/'
net_name              = 'MnistDifferentiableLut4Simple'
data_path             = os.path.join('./data/', net_name)
rtl_sim_path          = '../../verilog/mnist/tb_mnist_lut_simple'
rtl_module_name       = 'MnistLutSimple'
output_velilog_file   = os.path.join(data_path, rtl_module_name + '.v')
sim_velilog_file      = os.path.join(rtl_sim_path, rtl_module_name + '.v')

epochs                = 8
mini_batch_size       = 64
frame_modulation_size = 15

データセットは PyTorch の torchvision を使います。ミニバッチのサイズも DataLoader で指定しています。
BinaryBrainではミニバッチをフレーム数として FrameBufferオブジェクトで扱います。
バイナリ変調で計算中にフレーム数が変わるためデータセットの準備観点でのミニバッチと呼び分けています。

In [15]:
# dataset
dataset_path = './data/'
dataset_train = torchvision.datasets.MNIST(root=dataset_path, train=True, transform=transforms.ToTensor(), download=True)
dataset_test  = torchvision.datasets.MNIST(root=dataset_path, train=False, transform=transforms.ToTensor(), download=True)
loader_train = torch.utils.data.DataLoader(dataset=dataset_train, batch_size=mini_batch_size, shuffle=True, num_workers=2)
loader_test  = torch.utils.data.DataLoader(dataset=dataset_test,  batch_size=mini_batch_size, shuffle=False, num_workers=2)

## ネットワークの構築

DifferentiableLut に特に何もオプションをつけなければOKです。<br>
バイナリ変調を施すためにネットの前後に RealToBinary層とBinaryToReal層を入れています。<br>
send_command で "binary true" を送ることで、DifferentiableLut の内部の重み係数が 0.0-1.0 の間に拘束されます。

接続数がLUTの物理構成に合わせて、1ノード当たり6個なので層間で6倍以上ノード数が違うと接続されないノードが発生するので、注意してネットワーク設計が必要です。
最終段は各クラス7個の結果を出して Reduce で足し合わせています。こうすることで若干の改善がみられるとともに、加算結果が INT3 相当になるために若干尤度を数値的に見ることができるようです。

In [16]:
# define network
net = bb.Sequential([
            bb.RealToBinary(frame_modulation_size=frame_modulation_size),
            bb.DifferentiableLut([64*4*2], connection='random', N=4),
            bb.DifferentiableLut([64*4],   connection='serial', N=4),
            bb.DifferentiableLut([64],     connection='serial', N=4),
            bb.DifferentiableLut([10*4*4], connection='random', N=4),
            bb.DifferentiableLut([10*4],   connection='serial', N=4),
            bb.AverageLut       ([10],     connection='serial', N=4),
            bb.BinaryToReal(frame_integration_size=frame_modulation_size)
        ])
net.set_input_shape([1, 28, 28])

net.send_command("binary true")

loss      = bb.LossSoftmaxCrossEntropy()
metrics   = bb.MetricsCategoricalAccuracy()
optimizer = bb.OptimizerAdam(learning_rate=0.001)

optimizer.set_variables(net.get_parameters(), net.get_gradients())

## 学習の実施

load_networks/save_networks で途中結果を保存/復帰可能できます。ネットワークの構造が変わると正常に読み込めなくなるので注意ください。
(その場合は新しいネットをsave_networksするまで一度load_networks をコメントアウトください)

tqdm などを使うと学習過程のプログレス表示ができて便利です。

In [17]:
#bb.load_networks(data_path, net)

# learning
for epoch in range(epochs):
    # learning
    loss.clear()
    metrics.clear()
    with tqdm(loader_train) as t:
        for images, labels in t:
            x_buf = bb.FrameBuffer.from_numpy(images.numpy().astype(np.float32))
            t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[labels.numpy()].astype(np.float32))

            y_buf = net.forward(x_buf, train=True)
            
            dy_buf = loss.calculate(y_buf, t_buf)
            metrics.calculate(y_buf, t_buf)
            
            net.backward(dy_buf)

            optimizer.update()
            
            t.set_postfix(loss=loss.get(), acc=metrics.get())

    # test
    loss.clear()
    metrics.clear()
    for images, labels in loader_test:
        x_buf = bb.FrameBuffer.from_numpy(images.numpy().astype(np.float32))
        t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[labels.numpy()].astype(np.float32))

        y_buf = net.forward(x_buf, train=False)

        loss.calculate(y_buf, t_buf)
        metrics.calculate(y_buf, t_buf)

    print('epoch[%d] : loss=%f accuracy=%f' % (epoch, loss.get(), metrics.get()))

    bb.save_networks(data_path, net)

100%|██████████| 938/938 [00:04<00:00, 230.42it/s, acc=0.732, loss=1.71]


epoch[0] : loss=1.669669 accuracy=0.787200


100%|██████████| 938/938 [00:04<00:00, 234.22it/s, acc=0.775, loss=1.68]


epoch[1] : loss=1.693925 accuracy=0.760000


100%|██████████| 938/938 [00:03<00:00, 239.80it/s, acc=0.784, loss=1.67]


...

epoch[5] : loss=1.670191 accuracy=0.788900


100%|██████████| 938/938 [00:03<00:00, 245.95it/s, acc=0.788, loss=1.67]


epoch[6] : loss=1.700159 accuracy=0.750300


100%|██████████| 938/938 [00:03<00:00, 248.61it/s, acc=0.786, loss=1.67]


epoch[7] : loss=1.698916 accuracy=0.761900


## FPGA用Verilog出力

最後に学習したネットワークを Verilog 出力します。
MNISTのサイズである 28x28=784bit の入力を 10bit の分類をして出力するだけのシンプルなモジュールを出力します。

In [18]:
# export verilog
bb.load_networks(data_path, net)

# 結果を出力
with open(output_velilog_file, 'w') as f:
    f.write('`timescale 1ns / 1ps\n\n')
    bb.dump_verilog_lut_layers(f, module_name=rtl_module_name, net=net, device="")

# Simulation用ファイルに上書きコピー
os.makedirs(rtl_sim_path, exist_ok=True)
shutil.copyfile(output_velilog_file, sim_velilog_file)

'../../verilog/mnist/tb_mnist_lut_simple/MnistLutSimple.v'

In [19]:
# シミュレーション用データファイル作成
with open(os.path.join(rtl_sim_path, 'mnist_test.txt'), 'w') as f:
    bb.dump_verilog_readmemb_image_classification(f ,loader_test)

## モデルの内部の値を取得する

Verilog以外の言語やFPGA以外に適用したい場合、接続とLUTテーブルの2つが取得できれば同じ計算をするモデルをインプリメントすることが可能です。

### 事前準備
そのままだと勾配はリセットされているので少しだけ逆伝搬を実施します

In [20]:
# 最新の保存データ読み込み
bb.load_networks(data_path, net)

# layer を取り出す
layer0 = net[1]
layer1 = net[2]
layer2 = net[3]

### 接続を取得する

LUTモデルは get_connection_list() にて接続行列を取得できます。<br>
ここでの各出力ノードは、4つの入力と接続されており、layer0 の出力ノードは 256 個あるので、256x4 の行列が取得できます。

In [21]:
connection_mat = np.array(layer0.get_connection_list())
print(connection_mat.shape)
connection_mat

(512, 4)


array([[531, 302, 316, 420],
       [736, 111,  73, 191],
       [529, 203, 400,  20],
       ...,
       [770, 570, 749, 161],
       [633, 361, 153, 380],
       [221, 664, 145, 316]])

### FPGA化する場合のLUTテーブルを取得する

LUT化する場合のテーブルを取得します。<br>
4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
モデル内に BatchNormalization 等を含む場合はそれらも加味して最終的にバイナリLUTにする場合に適した値を出力します。

In [22]:
lut_mat = np.array(layer0.get_lut_table_list())
print(lut_mat.shape)
lut_mat

(512, 16)


array([[False, False, False, ...,  True, False, False],
       [False, False, False, ...,  True,  True,  True],
       [ True, False,  True, ..., False,  True,  True],
       ...,
       [False, False,  True, ..., False, False, False],
       [ True,  True,  True, ..., False,  True,  True],
       [ True,  True,  True, ...,  True, False, False]])

### 重み行列を覗いてみる

4入力のLUTモデルなので $ 2^4 = 16 $ 個のテーブルがあります。<br>
W() にて bb.Tensor 型で取得可能で、numpy() にて ndarray に変換できます。

In [23]:
W = layer0.W().numpy()
print(W.shape)
W

(512, 16)


array([[0.5104773 , 0.5037312 , 0.46286818, ..., 0.5631818 , 0.425866  ,
        0.41123012],
       [0.47676846, 0.41237223, 0.47465366, ..., 0.50881076, 0.57917786,
        0.5185155 ],
       [0.56667787, 0.49976224, 0.4749108 , ..., 0.4335499 , 0.57323796,
        0.61156875],
       ...,
       [0.44860807, 0.36814407, 0.60939187, ..., 0.46650785, 0.4733826 ,
        0.46093163],
       [0.46322957, 0.5300449 , 0.541812  , ..., 0.44569674, 0.51303834,
        0.4494719 ],
       [0.47722825, 0.55092996, 0.5285924 , ..., 0.5467777 , 0.33245566,
        0.41034847]], dtype=float32)

### 勾配を覗いてみる

同様に dW() でW の勾配が取得できます

In [24]:
# そのままだとすべて0なので、1回だけbackward実施
for images, labels in loader_test:
    x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
    t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))
    y_buf = net.forward(x_buf, train=True)
    net.backward(loss.calculate(y_buf, t_buf))
    break

dW = layer0.dW().numpy()
print(dW.shape)
dW

(512, 16)


/tmp/ipykernel_23608/4074597919.py:3: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  x_buf = bb.FrameBuffer.from_numpy(np.array(images).astype(np.float32))
/tmp/ipykernel_23608/4074597919.py:4: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments.
  t_buf = bb.FrameBuffer.from_numpy(np.identity(10)[np.array(labels)].astype(np.float32))


array([[-1.5820125e-03, -5.2733754e-04, -8.6357497e-05, ...,
         8.8202032e-05,  2.9151788e-04,  9.7172619e-05],
       [ 3.4756027e-04,  1.1585443e-04,  1.1585443e-04, ...,
        -3.8616534e-05, -3.8616534e-05, -1.2872100e-05],
       [ 2.8093492e-03,  9.1171166e-04,  7.1519415e-04, ...,
        -1.4825833e-04, -7.0538890e-04, -3.0934444e-04],
       ...,
       [ 5.1119237e-04,  1.7974142e-04,  4.3064813e-04, ...,
        -4.7264770e-05, -1.7319064e-04, -5.7384132e-05],
       [-2.3113291e-03,  6.7932019e-04, -7.6343073e-04, ...,
         1.0727937e-03,  3.5548280e-04,  3.7863478e-04],
       [-1.4529703e-04, -4.8432266e-05, -7.3652249e-05, ...,
         2.8921655e-05, -1.3782752e-05, -4.5943289e-06]], dtype=float32)